# <center>CS568:Deep Learning</center>  <center>Fall 2020</center> 
## Recitation 6 - Test 6  



**Dataset**: HAR UniMiB (2 classes)

**Training accuracy:** 99.54

**Testing accuracy:** 98.95

**Epochs**: 10

**Batch_size:**  32

**Loss function:** binary_crossentropy





In [29]:
import cv2
from numpy import mean
from numpy import std
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.mixture import GaussianMixture

In [30]:
### add code here

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Change the labels from categorical to one-hot encoding
train_Y = np.load("/content/drive/MyDrive/Colab Notebooks/pattern/dataset/two_classes_labels.npy")
labels  = train_Y[:,0]-1
train_Y_one_hot = to_categorical(labels)
#numpy file
np.save("/content/drive/MyDrive/Colab Notebooks/pattern/dataset/Binary_class_labels.npy",train_Y_one_hot)


In [32]:
train_Y_one_hot[0]

array([1., 0.], dtype=float32)

In [34]:
data =np.load("/content/drive/MyDrive/Colab Notebooks/pattern/dataset/two_classes_data.npy")
x_axis_data = data[:,:151]
y_axis_data = data[:,151:302]
z_axis_data = data[:,302:453]
    
    
#labels
Labels =np.load("/content/drive/MyDrive/Colab Notebooks/pattern/dataset/Binary_class_labels.npy")

In [35]:
def sliding_window(data):
    window_size = 40
    stride = 10
    return np.lib.stride_tricks.sliding_window_view(data,window_size)[::stride,:]

In [36]:
def GMM(data):
    gmm = GaussianMixture(n_components=12, covariance_type='spherical').fit(data)
    return gmm.means_

In [37]:
def find_NearestNeighbour(localdes,neighborhood):
    """
    Finds the nearest neighborhood of a vector.
    Args:
        point (float array): The initial point.
        neighborhood (numpy float matrix): The points that are around the initial point.
    Returns:
        float array: The point that is the nearest neighbor of the initial point.
        integer: Index of the nearest neighbor inside the neighborhood list
    """
    min_distance = float('inf')
    NearestNeighbour = neighborhood[0]
    NearestN_index = 0
    for i in range(len(neighborhood)):
        neighbor = neighborhood[i]
        distance = cv2.norm(localdes - neighbor)
        if distance < min_distance:
            min_distance = distance
            NearestNeighbour = neighbor
            NearestN_index = i 
    return NearestNeighbour, NearestN_index          

In [38]:
def vlad (localdescriptors, centers):
    """
    Calculate the Vector of Locally Aggregated Descriptors (VLAD) which is a global descriptor from a group of
    descriptors and centers that are codewords of a codebook, obtained for example with K-Means.
    Args:
        descriptors (numpy float matrix): The local descriptors.
        centers (numpy float matrix): The centers are points representatives of the classes.
    Returns:
        numpy float array: The VLAD vector.
    """    
        
    dimensions = len(localdescriptors[0])
    vlad_vector = np.zeros((len(centers), dimensions), dtype=np.float32)
    for localdescriptors in localdescriptors:
        nearest_center, center_index = find_NearestNeighbour(localdescriptors,centers)
        for i in range(dimensions):
            vlad_vector[center_index][i] += (localdescriptors[i] - nearest_center[i])
    # L2 Normalization            
    vlad_vector = cv2.normalize(vlad_vector,vlad_vector)
    vlad_vector = vlad_vector.flatten()
    return vlad_vector   
        

In [39]:
xfeaturevector = np.zeros((11771,480))
for i in range(x_axis_data.shape[0]):
    #sliding window in every row
    slide_vec = sliding_window(x_axis_data[i])
    #GMM in slide_vec shape 12X30
    codebook = GMM(slide_vec)
    #vlad implement retirn 480 dimensions
    v = vlad(slide_vec,codebook) 
    xfeaturevector[i] = v


In [40]:
yfeaturevector = np.zeros((11771,480))
for i in range(y_axis_data.shape[0]):
    #sliding window in every row
    slide_vec = sliding_window(y_axis_data[i])
    #GMM in slide_vec shape 12X30
    codebook = GMM(slide_vec)
    #vlad implement retirn 480 dimensions
    v = vlad(slide_vec,codebook) 
    yfeaturevector[i] = v


In [41]:
zfeaturevector =np.zeros((11771,480))
for i in range(z_axis_data.shape[0]):
    #sliding window in every row
    slide_vec = sliding_window(z_axis_data[i])
    #GMM in slide_vec shape 12X30
    codebook = GMM(slide_vec)
    #vlad implement retirn 480 dimensions
    v = vlad(slide_vec,codebook) 
    zfeaturevector[i] = v


In [42]:
final_feature_vector = np.concatenate((xfeaturevector,yfeaturevector,zfeaturevector),axis=1)
final_feature_vector.shape

(11771, 1440)

In [43]:
def load_dataset():
  train_data, test_data, train_labels, test_labels = train_test_split(final_feature_vector, Labels, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=0)
  print("Labels for training and testing data")
  print(len(train_data))
  print(len(test_data))
  return train_data, train_labels, test_data, test_labels

In [44]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 10, 32
  n_outputs = 2
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(1440,1)))
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  model.add(Dropout(0.5))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy, model

In [45]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [46]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score ,model  = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)
	return model

In [47]:
# run the experiment
model = run_experiment()

Labels for training and testing data
8239
3532
>#1: 97.961
>#2: 98.754
>#3: 98.641
>#4: 98.783
>#5: 98.358
>#6: 98.867
>#7: 97.707
>#8: 98.698
>#9: 98.726
>#10: 98.952
[97.96149730682373, 98.7542450428009, 98.64099621772766, 98.78255724906921, 98.35786819458008, 98.86749982833862, 97.70668148994446, 98.69762063026428, 98.72593283653259, 98.95243644714355]
Accuracy: 98.545% (+/-0.389)


In [48]:
# training accuracy 
import torch
total = 0
correct = 0
trainX, trainy, testX, testy = load_dataset()
inputs = trainX
targets = trainy
_, accuracy = model.evaluate(trainX, trainy, batch_size=32, verbose=0)
        
print("Accuracy on training data: ", accuracy*100)


# testing accuracy
total = 0
correct = 0
inputs = testX
targets = testy
_, accuracy = model.evaluate(testX, testy, batch_size=32, verbose=0)
        
print("Accuracy on testing data: ", accuracy*100)

Labels for training and testing data
8239
3532
Accuracy on training data:  99.84221458435059
Accuracy on testing data:  98.95243644714355
